In [1]:
import requests

In [2]:
api_url="https://CrimsonIncompleteDebuggers.diarrassoubasak.repl.co"

In [3]:
response=requests.get(api_url + '/ping')


In [4]:
response.text

'pong'

# Envoyer des lignes du dataset de Test

In [5]:
test_url="https://bit.ly/titanic-test-set"

import pandas as pd

df=pd.read_csv(test_url,index_col='PassengerId')

In [6]:
df.head(3)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


# Isoler une ligne du dataset

In [7]:
df.iloc[0]

Pclass                     3
Name        Kelly, Mr. James
Sex                     male
Age                     34.5
SibSp                      0
Parch                      0
Ticket                330911
Fare                  7.8292
Cabin                    NaN
Embarked                   Q
Name: 892, dtype: object

Du coté de mon API (model), notre modele fait des prédictions à partir du dataframe (objet 2D), mais la serie qu'on va envoyer est une serie (objet 1D)

In [29]:
# Envoie (Client --> API)
payload=df.iloc[[0]].to_json()

In [30]:
payload

'{"Pclass":{"892":3},"Name":{"892":"Kelly, Mr. James"},"Sex":{"892":"male"},"Age":{"892":34.5},"SibSp":{"892":0},"Parch":{"892":0},"Ticket":{"892":"330911"},"Fare":{"892":7.8292},"Cabin":{"892":null},"Embarked":{"892":"Q"}}'

In [31]:
# Reception ( dans le code de notre API)
pd.read_json(payload,typ='frame',orient='columns')

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


In [35]:
resultat=requests.post(api_url + '/predict',
                  data=df.iloc[[12]].to_json(),
                  headers={'Content-Type':'application/json'})

In [36]:
resultat

<Response [201]>

In [37]:
resultat.text

'1'

In [38]:
url_heroku="https://titanic-app-api-2020.herokuapp.com"

In [39]:
response=requests.get(url_heroku + '/ping')

response, response.text

(<Response [200]>, 'pong')

In [40]:
def predict_api(df,idx):
  payload=df.iloc[[idx]].to_json()
  url_heroku="https://titanic-app-api-2020.herokuapp.com"

  response= requests.post(url_heroku + '/predict',
                         data=payload ,
                         headers={'Content-Type':'application/json'})
  #print(response, idx)
  return (int(response.text))


In [41]:
predict_api(df,12)

<Response [201]> 12
1


In [46]:
def predict_api_titanic(df,idx):
  
  passenger_id=df.iloc[idx].name

  # Envoyer à heroku
  payload=df.iloc[[idx]].to_json()
  url_heroku="https://titanic-app-api-2020.herokuapp.com"

  response= requests.post(url_heroku + '/predict',
                         data=payload ,
                         headers={'Content-Type':'application/json'})
  # Formatte les resultats  dans un dico 
  return {'PassengerId' : passenger_id, 'Survived': int(response.text)}


In [47]:
predict_api_titanic(df,0)

{'PassengerId': 892, 'Survived': 0}

In [48]:
df.shape

(418, 10)

In [49]:
resultat_final=[predict_api_titanic(df,idx) for idx in range(418)]

In [53]:
pd.DataFrame(resultat_final).to_csv('Soumission_kaggle.csv', index=False)